In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
gK=os.getenv('GROQ_API_KEY')

In [4]:
llm = ChatGroq(model='Llama3-8b-8192',groq_api_key=gK)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FFBD91E9C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FFBD91F500>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
# load data source ->chunk ->embedding ->stores to vector 
# chathistory
#      ->prompt - context prompt
#  - chat history 
# -----------------------

In [6]:
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN') 
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [10]:
!pip install langchain_chroma

  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached chromadb-0.5.20-py3-none-any.whl.metadata (6.8 kB)
  Using cached fastapi-0.115.5-py3-none-any.whl.metadata (27 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached uvicorn-0.32.1-py3-none-any.whl.metadata (6.6 kB)
  Using cached posthog-3.7.2-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached opentelemetry_api-1.28.2-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.28.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentatio

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires protobuf<5,>=3.20, but you have protobuf 5.28.3 which is incompatible.
streamlit 1.32.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.


In [14]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [16]:
import bs4
WebBaseLoader(web_path=('https://lilianweng.github.io/posts/2023-06-23-agent/'),bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-title","post-content","post-header")),))

In [22]:
import bs4
loader=WebBaseLoader(web_path=('https://lilianweng.github.io/posts/2023-06-23-agent/'),bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-title","post-content","post-header")),))
docs = loader.load()

In [24]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits,embedding=embeddings)
retriver = vectorstore.as_retriever()
retriver

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001FFCF862E40>, search_kwargs={})

In [72]:
## Prompt Template
system_prompt = ("You are as assistant for question-answering tasks"
                 "\n\n"
                 "{context}")
prompt = ChatPromptTemplate.from_messages([("system",system_prompt),("human","{input}"),])

In [74]:
qa_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriver,qa_chain)

In [52]:
response = rag_chain.invoke({"input":"what is Self-Reflection"})
#print(response)

response['answer']

"According to the text, Self-Reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable. In the Reflexion framework, Self-Reflection is created by showing two-shot examples to LLM and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Then reflections are added into the agent's working memory, up to three, to be used as context for querying LLM."

In [80]:
response = rag_chain.invoke({"input":"how do we move fast"})
response['answer']
#print(response)

"To move fast in an LLM-powered autonomous agent system, you can focus on the following strategies:\n\n1. **Task Decomposition**: Break down complex tasks into smaller, manageable subgoals. This helps the agent to prioritize and focus on the most important steps, reducing the overall time and effort required to complete the task.\n2. **Reflection and Refinement**: Regularly reflect on past actions and decisions, and refine the plan accordingly. This self-critical approach helps the agent to learn from mistakes and adapt to changing circumstances, enabling faster progress.\n3. **Efficient Planning**: Use the heuristic function to identify and stop inefficient planning trajectories. This ensures that the agent doesn't waste time on unnecessary or repetitive actions.\n4. **Smart Task Delegation**: Leverage the power of GPT-3.5 to delegate simple tasks and focus on more complex and high-priority tasks.\n5. **Contextual Querying**: Use the agent's working memory to query LLM with contextual

In [84]:
## Prompt Template
system_prompt = ("You are as assistant for question-answering tasks"
                 "Keep the answer concise"
                 "If you don't know the answer,say don't know"
                 "\n\n"
                 "{context}")
prompt = ChatPromptTemplate.from_messages([("system",system_prompt),("human","{input}"),])

In [86]:
qa_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriver,qa_chain)

In [88]:
response = rag_chain.invoke({"input":"how do we move fast"})
response['answer']

'To move fast, the LLM-powered autonomous agent system uses the following strategies:\n\n1. **Task decomposition**: Breaking down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\n2. **Reflection and refinement**: The agent can do self-criticism and self-reflection over past actions, learn from mistakes, and refine them for future steps, thereby improving the quality of final results.\n3. **Smart and efficient planning**: Continuously reviewing and analyzing actions, constructively self-criticizing big-picture behavior, reflecting on past decisions and strategies, and aiming to complete tasks in the least number of steps.\n4. **Heuristic function**: Stopping inefficient or hallucinating trajectories to prevent unnecessary exploration and optimize the planning process.\n\nBy incorporating these strategies, the agent can move quickly and efficiently towards achieving its goals.'

In [68]:
chain = prompt|llm
chain.invoke({"input":"what is QA","context":"Msg"})

AIMessage(content='QA stands for Question Answering, which is a task in which a system or model is designed to answer questions based on the information it has been trained on. The goal is to provide accurate and relevant responses to user queries, which can be in the form of natural language text, voice commands, or other inputs.\n\nIn the context of artificial intelligence and natural language processing, QA is a common application of machine learning and deep learning techniques. The system is trained on a large dataset of questions and answers, and then uses this knowledge to generate responses to new, unseen questions.\n\nThere are different types of QA, including:\n\n1. Open-domain QA: This type of QA involves answering questions from a wide range of topics and domains.\n2. Closed-domain QA: This type of QA involves answering questions within a specific domain or topic.\n3. Conversational QA: This type of QA involves answering questions in a conversational manner, often with foll

In [70]:
chain.invoke({"input":"how do we move fast","context":"Msg"})

AIMessage(content="There are several ways to move quickly, depending on the context and the situation. Here are a few examples:\n\n1. Running: One of the fastest ways to move is by running. You can jog, sprint, or even do short bursts of speed to move quickly.\n2. Cycling: If you have access to a bike, cycling is another great way to move quickly. You can ride at a moderate pace or sprint if you need to.\n3. Swimming: Swimming is a fast and efficient way to move through water. You can do the breaststroke, freestyle, or even try some more advanced strokes like the butterfly.\n4. Skateboarding or rollerblading: If you're near a skatepark or have access to a skateboard or rollerblades, you can use these to move quickly and have fun.\n5. Taking a car or public transportation: If you don't have time to move on foot or by bike, you can take a car or use public transportation like a bus or train to get where you need to go quickly.\n\nI hope these suggestions are helpful!", additional_kwargs=

In [94]:
## Adding Chat History
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

q_system_prompt = ( "Given a chat history and latest user question"
                   "which might reference context in the chat history"
                   "without the chat history.Do NOT answer the question" )
q_prompt = ChatPromptTemplate.from_messages( 
    [ ("system",q_system_prompt),MessagesPlaceholder("chat_history"),
("human","{input}"),])

In [102]:
#help(create_history_aware_retriever)
history_aware_retriever = create_history_aware_retriever(llm,retriver,q_prompt)

In [106]:
qa_prompt = ChatPromptTemplate.from_messages( [("system",system_prompt),
                                               MessagesPlaceholder("chat_history"),
                                               ("human","{input}"),])

In [110]:
qa_chain = create_stuff_documents_chain(llm,qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever,qa_chain)

In [114]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history = list()
question="what is Self-Reflection"
response1 = rag_chain.invoke({"input":question,"chat_history":chat_history})
response1

{'input': 'what is Self-Reflection',
 'chat_history': [],
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 3. Illustration of the Reflexion framework. (Image source: Shinn & Labash, 2023)\nThe heuristic function determines when the trajectory is inefficient or contains hallucination and should be stopped. Inefficient planning refers to trajectories that take too long without success. Hallucination is defined as encountering a sequence of consecutive identical actions that lead to the same observation in the environment.\nSelf-reflection is created by showing two-shot examples to LLM and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Then reflections are added into the agent’s working memory, up to three, to be used as context for querying LLM.'),
  Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Another quite d

In [118]:
response1 = rag_chain.invoke({"input":"what is llm","chat_history":chat_history})
response1

{'input': 'what is llm',
 'chat_history': [],
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='API-Bank (Li et al. 2023) is a benchmark for evaluating the performance of tool-augmented LLMs. It contains 53 commonly used API tools, a complete tool-augmented LLM workflow, and 264 annotated dialogues that involve 568 API calls. The selection of APIs is quite diverse, including search engines, calculator, calendar queries, smart home control, schedule management, health data management, account authentication workflow and more. Because there are a large number of APIs, LLM first has access to API search engine to find the right API to call and then uses the corresponding documentation to make a call.'),
  Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='(4) Response generation: LLM receives the execution results and provides summarized results to users.\nTo put HuggingGPT into

In [120]:
chat_history.extend([HumanMessage(content=question),AIMessage(content=response1["answer"])])

In [122]:
question2="what is api?"
response2 = rag_chain.invoke({"input":question2,"chat_history":chat_history})
print(response2["answer"])

{
    "thoughts": {
        "text": "An API, or Application Programming Interface, is a set of defined rules that enable different applications to communicate with each other.",
        "reasoning": "APIs allow different systems to exchange data and functionality, enabling integration and interaction between them.",
        "plan": "- APIs provide a standardized way for applications to request and receive data.",
        "criticism": "APIs can be complex and require careful consideration of security and data management.",
        "speak": "APIs are a fundamental tool for building modern software systems."
    },
    "command": {
        "name": "define",
        "args": {}
    }
}


In [124]:
chat_history

[HumanMessage(content='what is Self-Reflection', additional_kwargs={}, response_metadata={}),
 AIMessage(content='LLM stands for Large Language Model. It refers to a type of artificial intelligence (AI) model that is trained on a massive amount of text data to generate human-like language outputs. LLMs are capable of understanding and generating natural language, and are often used for tasks such as language translation, text summarization, and chatbots.', additional_kwargs={}, response_metadata={})]

In [126]:
response2 = rag_chain.invoke({"input":question,"chat_history":chat_history})
print(response2["answer"])

Don't know.


In [128]:
chat_history

[HumanMessage(content='what is Self-Reflection', additional_kwargs={}, response_metadata={}),
 AIMessage(content='LLM stands for Large Language Model. It refers to a type of artificial intelligence (AI) model that is trained on a massive amount of text data to generate human-like language outputs. LLMs are capable of understanding and generating natural language, and are often used for tasks such as language translation, text summarization, and chatbots.', additional_kwargs={}, response_metadata={})]

In [130]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id:str) ->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(rag_chain,
                           get_session_history,
                           input_messages_key="input",
                           history_messages_key="chat_history",
                           output_messages_key="answer",)

In [132]:
r1 = conversational_rag_chain.invoke({"input":"what is quantization?"},
                               config={"configurable":{"session_id":"h1"}})
r1["answer"]

'Quantization is the process of mapping a continuous-valued input signal to a finite discrete set of values, typically represented by a fixed number of bits. In the context of FAISS and ScaNN, vector quantization is used to partition a high-dimensional vector space into clusters and then refine the quantization within each cluster. The goal is to reduce the dimensionality of the data while preserving its essential characteristics.'

In [134]:
r2 = conversational_rag_chain.invoke({"input":"how to move fast"},
                               config={"configurable":{"session_id":"h1"}})
r2["answer"]

"I don't know"

In [136]:
store

{'h1': InMemoryChatMessageHistory(messages=[HumanMessage(content='what is quantization?', additional_kwargs={}, response_metadata={}), AIMessage(content='Quantization is the process of mapping a continuous-valued input signal to a finite discrete set of values, typically represented by a fixed number of bits. In the context of FAISS and ScaNN, vector quantization is used to partition a high-dimensional vector space into clusters and then refine the quantization within each cluster. The goal is to reduce the dimensionality of the data while preserving its essential characteristics.', additional_kwargs={}, response_metadata={}), HumanMessage(content='how to move fast', additional_kwargs={}, response_metadata={}), AIMessage(content="I don't know", additional_kwargs={}, response_metadata={})])}

In [140]:
### PyPDFDirectoryLoader
import os
from langchain_groq import ChatGroq
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [142]:
from dotenv import load_dotenv
load_dotenv()

True

In [144]:
## load the Groq API Key
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
gk=os.getenv('GROQ_API_KEY')

In [152]:
#embeddings = OllamaEmbeddings(model="gemma:2b")
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

In [158]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

In [162]:
llm = ChatGroq(model='Llama3-8b-8192',groq_api_key=gk)

In [164]:
prompt=ChatPromptTemplate.from_template(
    '''
    Answer the question based on the provided context only.
    provide the most accurate respone based on question
    <context>
    {context}
    <context>
    Question:{input}''')
    

In [166]:
loader = PyPDFDirectoryLoader("research_papers")
docs = loader.load()
text=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_docs = text.split_documents(docs[:50])
FAISS.from_documents(final_docs,embedding)

doc_chain = create_stuff_documents_chain(llm.prompt)
retriver = vectors.as_retrivever()
retrieval_chain = create_retrieval_chain(retriver,doc_chain)
response = retrieval_chain.invoke({'input':user_prompt})
response['answer']

In [170]:
loader = PyPDFDirectoryLoader("research_papers")
loader

In [174]:
L=['Data1','Data2','Data3']

for var in enumerate(L):
    print(var)

(0, 'Data1')
(1, 'Data2')
(2, 'Data3')


In [176]:
for v1,v2 in enumerate(L):
    print(v2)

Data1
Data2
Data3


In [ ]:
## Text Summarization

In [180]:
api_key = os.getenv("GROQ_API_KEY")

In [182]:
llm=ChatGroq(groq_api_key=api_key,model="Gemma-7b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FFE317DC10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FFE317F710>, model_name='Gemma-7b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [186]:
speech = """The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.
Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.
We have borne with their present government through all these bitter months because of that friendshipâ€”exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.
It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free."""

In [188]:
llm.get_num_tokens(speech)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

C:\Users\theeba\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\theeba\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

675

In [192]:
from langchain.schema import (AIMessage,HumanMessage,SystemMessage)

In [194]:
chat_message=[SystemMessage(content="Give Summarizing speech"),
              HumanMessage(content=f"Please provide short concisse summary")]

In [198]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [196]:
llm(chat_message)

C:\Users\theeba\AppData\Local\Temp\ipykernel_11488\2694584740.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm(chat_message)


AIMessage(content="**Summary Speech: The Importance of Education**\n\n**Introduction:**\n\nEducation is the foundation of a prosperous and just society. It empowers individuals, transforms communities, and drives economic growth. In today's rapidly changing world, access to quality education is more important than ever.\n\n**Body:**\n\n- **Cognitive Development:** Education fosters critical thinking, creativity, and problem-solving skills.\n- **Economic Opportunities:** Education opens doors to high-paying jobs and financial security.\n- **Social Transformation:** Education promotes social mobility and creates a more equitable society.\n- **Civic Engagement:** Education empowers individuals to become informed citizens and participate actively in democracy.\n- **Personal Growth:** Education fosters a lifelong love of learning and personal growth.\n\n**Conclusion:**\n\nEducation is an essential investment in our future. By investing in education, we empower individuals to create a better

In [208]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("LVMActivity.pdf")
docs = loader.load_and_split()
docs

[Document(metadata={'source': 'LVMActivity.pdf', 'page': 0}, page_content='Creating Linux LVM Partitions \nTasks \n1. Create two 1G partitions on /dev/xvdb. \na. Use the fdisk command to create two 1G primary partitions on /dev/xvdb as shown \nin the following: \n# fdisk /dev/xvdb \nWelcome to dfisk (util-linux 2.23.2). \n... \nCommand (m for help): n \nPartition type: \np primary partition (0 primary, 0 extended, 4 free) \ne extended \nSelect (default p): ENTER \nUsing default response p \nPartition number (1-4, default 1): ENTER \nFirst sector (2048-10485759, default 2048): ENTER \nUsing default value 2048 \nLast sector, +sectors or +size{K,M,G} (2048-10485759, default \n10485759): +1G \nPartition 1 of type Linux and of size 1 GiB is set \nCommand (m for help): n \nPartition type: \np primary partition (1 primary, 0 extended, 3 free) \ne extended \nSelect (default p): ENTER \nUsing default response p \nPartition number (2-4, default 2): ENTER \nFirst sector (2099200-10485759, default

In [214]:
from langchain import PromptTemplate

In [216]:
template = '''Write a concise and short summary about: {text}'''

prompt =PromptTemplate(input_variables=['text'],template=template)

In [218]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm,chain_type='stuff',prompt=prompt,verbose=True)
output_summary = chain.run(docs)
output_summary

C:\Users\theeba\AppData\Local\Temp\ipykernel_11488\3940093373.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output_summary = chain.run(docs)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise and short summary about: Creating Linux LVM Partitions 
Tasks 
1. Create two 1G partitions on /dev/xvdb. 
a. Use the fdisk command to create two 1G primary partitions on /dev/xvdb as shown 
in the following: 
# fdisk /dev/xvdb 
Welcome to dfisk (util-linux 2.23.2). 
... 
Command (m for help): n 
Partition type: 
p primary partition (0 primary, 0 extended, 4 free) 
e extended 
Select (default p): ENTER 
Using default response p 
Partition number (1-4, default 1): ENTER 
First sector (2048-10485759, default 2048): ENTER 
Using default value 2048 
Last sector, +sectors or +size{K,M,G} (2048-10485759, default 
10485759): +1G 
Partition 1 of type Linux and of size 1 GiB is set 
Command (m for help): n 
Partition type: 
p primary partition (1 primary, 0 extended, 3 free) 
e extended 
Select (default p): ENTER 
Using default response p 
Partition number (2-4, default 2): E

'**Creating Linux LVM Partitions and Volumes**\n\n**Step 1: Create Partitions**\n\n- Created two 1 GB partitions on both /dev/xvdb and /dev/xvdd.\n- Changed the system ID of the second partition on each disk to "8e" (Linux LVM).\n\n**Step 2: Initialize Physical Volumes**\n\n- Used the `pvcreate` command to create physical volumes on both partitions.\n\n**Step 3: Create Volume Group**\n\n- Created a volume group named "myvolg" using the `vgcreate` command.\n\n**Step 4: Create Logical Volume**\n\n- Created a logical volume named "myvol" within the "myvolg" volume group.\n\n**Step 5: Format and Mount Logical Volume**\n\n- Formatted the logical volume with the ext4 file system.\n- Created a mount point and mounted the logical volume.\n\n**Step 6: Create Thinly Provisioned Logical Volumes**\n\n- Created a thin pool and two thinly provisioned volumes.\n\n**Step 7: Create Filesystems and Mount**\n\n- Created Ext4 file systems on the thinly provisioned volumes.\n- Mounted the volumes to /myvol

In [220]:
# Map reduce to summarize large docs
from langchain.text_splitter import RecursiveCharacterTextSplitter
final_docs = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100).split_documents(docs)

In [222]:
len(final_docs)

22

In [224]:
chunk_prompt="""
Please summarize the below text
Content:'{text}'
Summary:"""
map_prompt_template = PromptTemplate(input_variables=['text'],template=chunk_prompt)


In [226]:
final_prompt='''
Provide final summary of the entir doc 
Content:{text}'''
final_prompt_template = PromptTemplate(input_variables=['text'],template=final_prompt)
final_prompt_template

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nProvide final summary of the entir doc \nContent:{text}')

In [228]:
summary_chain  = load_summarize_chain(llm=llm,chain_type="map_reduce",
                                      map_prompt=map_prompt_template,
                                      combine_prompt=final_prompt_template,
                                      verbose=True)
output=summary_chain.run(final_docs)
output



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below text
Content:'Creating Linux LVM Partitions 
Tasks 
1. Create two 1G partitions on /dev/xvdb. 
a. Use the fdisk command to create two 1G primary partitions on /dev/xvdb as shown 
in the following: 
# fdisk /dev/xvdb 
Welcome to dfisk (util-linux 2.23.2). 
... 
Command (m for help): n 
Partition type: 
p primary partition (0 primary, 0 extended, 4 free) 
e extended 
Select (default p): ENTER 
Using default response p 
Partition number (1-4, default 1): ENTER 
First sector (2048-10485759, default 2048): ENTER 
Using default value 2048 
Last sector, +sectors or +size{K,M,G} (2048-10485759, default 
10485759): +1G 
Partition 1 of type Linux and of size 1 GiB is set 
Command (m for help): n 
Partition type: 
p primary partition (1 primary, 0 extended, 3 free) 
e extended 
Select (default p): ENTER 
Using default response p 
Partition number (2-4, default 

Token indices sequence length is longer than the specified maximum sequence length for this model (4053 > 1024). Running this sequence through the model will result in indexing errors



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Provide final summary of the entir doc 
Content:**Summary:**

The provided text outlines the steps to create two 1 gigabyte partitions on the /dev/xvdb storage device using the fdisk command.

**Steps:**

1. Launch the fdisk command to interact with the partition table.


2. Create the first 1 gigabyte primary partition:
    - Select `p` for primary partition.
    - Assign partition number 1.
    - Set the first sector to 2048 and the last sector to +1G.


3. Create the second 1 gigabyte primary partition:
    - Select `p` for primary partition.
    - Assign partition number 2.
    - Set the first sector to 2099200 and the last sector to +1G.


4. Verify that the partitions have been created successfully.

**Summary:**

The text provides instructions for partitioning a disk and setting up a Linux system.

**Key steps:**

1. **Enter the first sector:** 2099200
2.

'## Final Summary:\n\nThe provided document details the process of managing **thin volumes and thin pools** using the LVM2 command-line interface in a Linux system.\n\n**Key Findings:**\n\n- Creating a thin pool with a size of 100 MB.\n- Identifying thin volumes using the `lvs` command.\n- Data percentage values for existing volumes were 0.00.\n- The document highlights the benefits of using thin volumes for flexible and efficient storage management.\n\n**Overall, the document serves as a comprehensive guide to understanding and implementing thin volume management in a Linux environment.**'

In [230]:
summary_chain  = load_summarize_chain(llm=llm,chain_type="refine",verbose=True)
output=summary_chain.run(final_docs)
output



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Creating Linux LVM Partitions 
Tasks 
1. Create two 1G partitions on /dev/xvdb. 
a. Use the fdisk command to create two 1G primary partitions on /dev/xvdb as shown 
in the following: 
# fdisk /dev/xvdb 
Welcome to dfisk (util-linux 2.23.2). 
... 
Command (m for help): n 
Partition type: 
p primary partition (0 primary, 0 extended, 4 free) 
e extended 
Select (default p): ENTER 
Using default response p 
Partition number (1-4, default 1): ENTER 
First sector (2048-10485759, default 2048): ENTER 
Using default value 2048 
Last sector, +sectors or +size{K,M,G} (2048-10485759, default 
10485759): +1G 
Partition 1 of type Linux and of size 1 GiB is set 
Command (m for help): n 
Partition type: 
p primary partition (1 primary, 0 extended, 3 free) 
e extended 
Select (default p): ENTER 
Using default response p 
Partition number (2-4, default 

'## Creating and Managing Linux LVM Partitions\n\n**Creating and Managing LVM Partitions and Volumes:**\n\nCreating LVM partitions and volumes involves partitioning a storage device, creating physical volumes, forming a volume group, and creating logical volumes from it.\n\n**Verifying and Managing LVM Configuration:**\n\nTools like `vgs`, `lvdisplay`, `lvs`, and `lvscan` help verify the volume group status and detailed information about logical volumes.\n\n**Creating Thin Pools and Volumes:**\n\nThin pools offer flexible storage management by allocating storage on demand. These thin volumes can be created from the thin pool.\n\n**Monitoring Thin Volume Usage:**\n\nThe `lvs` command provides accurate information about logical volumes, including their utilization. Be aware that the `df` command may show discrepancies due to the nature of thin provisioning.\n\n**Example Usage:**\n\nThe provided example demonstrates creating thin volumes and monitoring their usage by copying files and che

In [ ]:
Quantization
 |
 |-> HighMemory ->LowerMemory